<a href="https://colab.research.google.com/github/abogaziah/Pong_DQN/blob/master/Pong_DQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Replay buffer
Sotres a tuple of (State, action, reward, next state, termiante) every time step then samples it to the NN as a training set  

In [0]:
from collections import deque
import random

class buffer:

  def __init__(self, buffer_size):
    self.buffer_size = buffer_size
    self.count=0
    self.buffer=deque()

  def append(self,s,a,r,s_,t):
    step=(s,a,r,s_)
    if self.count < self.buffer_size:
      self.buffer.append(step)
      self.count+=1
    else:
      self.buffer.popleft()
      self.buffer.append(step)

  def sample(self,batch_size):
    batch=[]
    if self.count <= batch_size:
      batch=random.sample(self.buffer, self.count)
    else:
      batch=random.sample(self.buffer,batch_size)

    s, a, r, s_ ,t = list(map(np.array, list(zip(*batch))))
    return s, a, r, s_ , t

  def clear(self):
        self.buffer.clear()
        self.count = 0

In [0]:
import keras
import gym
import numpy as np
import cv2
from keras.models import load_model, Sequential
from keras.layers.convolutional import Convolution2D
from keras.optimizers import Adam
from keras.layers.core import Activation, Dropout, Flatten, Dense

In [0]:
lr=0.0001
buffer_size=1000000

In [0]:
def dqn(n_actions,input_shape,lr):
  model = Sequential()
  model.add(Conv2D(filters=32, kernel_size=8, strides=4, activation='relu',
                    input_shape=(*input_shape,), data_format='channels_first'))
  model.add(Conv2D(filters=64, kernel_size=4, strides=2, activation='relu',
                    data_format='channels_first'))
  model.add(Conv2D(filters=64, kernel_size=3, strides=1, activation='relu',
                    data_format='channels_first'))
  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dense(n_actions))

  model.compile(optimizer=Adam(lr=lr), loss='mean_squared_error')

  return model


In [0]:
class agent:
  def __init__(self, alpha, gamma, n_actions, epsilon, batch_size, replace,
                 input_dims, eps_dec=0.996,  eps_min=0.01,
                 mem_size=1000000, q_eval='q_eval.h5',
                 q_target='target.h5'):
        self.action_space = [i for i in range(n_actions)]
        self.gamma = gamma
        self.epsilon = epsilon
        self.eps_dec = eps_dec
        self.eps_min = eps_min
        self.batch_size = batch_size
        self.replace = replace
        self.q_target_model_file = q_target
        self.q_eval_model_file = q_eval
        self.learn_step = 0
        self.memory = buffer(mem_size)
        self.q_eval = build_dqn(alpha, n_actions, input_dims, 512)
        self.q_next = build_dqn(alpha, n_actions, input_dims, 512)